In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import datetime, os
spark = SparkSession.builder.appName("FileValidation").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/31 15:36:13 WARN Utils: Your hostname, Pulastyas-Mac-mini.local, resolves to a loopback address: 127.0.0.1; using 10.0.0.133 instead (on interface en1)
26/01/31 15:36:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/31 15:36:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = spark.read.text("data/input/data_with_ht.csv")\
    .withColumn("RecordType", F.split(F.col("value"), ",").getItem(0)) \
    .withColumn("Name", F.when(F.col("RecordType") == "D", F.split(F.col("value"), ",").getItem(1)).otherwise(None)) \
    .withColumn("Age", F.when(F.col("RecordType") == "D", F.split(F.col("value"), ",").getItem(2).cast("int")).otherwise(None)) \
    .withColumn("Gender", F.when(F.col("RecordType") == "D", F.split(F.col("value"), ",").getItem(3)).otherwise(None)) \
    .withColumn("Salary", F.when(F.col("RecordType") == "D", F.split(F.col("value"), ",").getItem(4).cast("int")).otherwise(None)) \
    .withColumn("FileName", F.when(F.col("RecordType") == "H", F.split(F.col("value"), ",").getItem(1)).otherwise(None)) \
    .withColumn("FileDate", F.when(F.col("RecordType") == "H", F.split(F.col("value"), ",").getItem(2)).otherwise(None)) \
    .withColumn("total_records", F.when(F.col("RecordType") == "T", F.split(F.col("value"), ",").getItem(1).cast("int")).otherwise(None)) \
    .withColumn("total_age", F.when(F.col("RecordType") == "T", F.split(F.col("value"), ",").getItem(2).cast("int")).otherwise(None)) \
    .withColumn("total_males", F.when(F.col("RecordType") == "T", F.split(F.col("value"), ",").getItem(3).cast("int")).otherwise(None)) \
    .withColumn("total_females", F.when(F.col("RecordType") == "T", F.split(F.col("value"), ",").getItem(4).cast("int")).otherwise(None)) \
    .withColumn("total_salary", F.when(F.col("RecordType") == "T", F.split(F.col("value"), ",").getItem(5).cast("int")).otherwise(None)) \
    .drop("value")


data.show(truncate=False)

+----------+------+----+------+------+----------------+--------+-------------+---------+-----------+-------------+------------+
|RecordType|Name  |Age |Gender|Salary|FileName        |FileDate|total_records|total_age|total_males|total_females|total_salary|
+----------+------+----+------+------+----------------+--------+-------------+---------+-----------+-------------+------------+
|H         |NULL  |NULL|NULL  |NULL  |data_with_ht.csv|20260130|NULL         |NULL     |NULL       |NULL         |NULL        |
|D         |Rahul |35  |M     |50000 |NULL            |NULL    |NULL         |NULL     |NULL       |NULL         |NULL        |
|D         |Anita |28  |F     |60000 |NULL            |NULL    |NULL         |NULL     |NULL       |NULL         |NULL        |
|D         |Sunil |40  |F     |70000 |NULL            |NULL    |NULL         |NULL     |NULL       |NULL         |NULL        |
|D         |Preeti|32  |F     |75000 |NULL            |NULL    |NULL         |NULL     |NULL       |NULL

In [ ]:
trailer = data.filter(F.col("RecordType") == "T")
detail = data.filter(F.col("RecordType") == "D")
header = data.filter(F.col("RecordType") == "H")

filedateinheader = header.select("FileDate").first()["FileDate"]
filenameinheader = header.select("FileName").first()["FileName"]

actual = detail.agg(F.count("*").alias("total_records"),
                    F.sum("Age").alias("total_age"),
                    F.sum(F.when(F.col("Gender") == "M", 1).otherwise(0)).alias("total_males"),
                    F.sum(F.when(F.col("Gender") == "F", 1).otherwise(0)).alias("total_females"),
                    F.sum("Salary").alias("total_salary")).first()

expected = trailer.select("total_records", "total_age", "total_males", "total_females", "total_salary").first()

current_date = datetime.datetime.now().strftime("%Y%m%d")

errors = []

if filedateinheader != current_date:
    errors.append(f"File date in header {filedateinheader} does not match expected date {current_date}")

if filenameinheader != "data_with_ht.csv":
    errors.append(f"File name in header {filenameinheader} does not match expected name data_with_ht.csv")

for key in expected.asDict().keys():
    if actual[key] != expected[key]:
        errors.append(f"Mismatch in {key}: actual={actual[key]}, expected={expected[key]}") 

if errors:
    print("File validation failed with following errors:")
    for error in errors:
        print(error)
else:
    print("File validation passed successfully.")

File validation failed with following errors:
File date in header 20260130 does not match expected date 20260131
Mismatch in total_age: actual=181, expected=180
Mismatch in total_males: actual=2, expected=3
Mismatch in total_females: actual=3, expected=2
Mismatch in total_salary: actual=335000, expected=325000


26/01/31 20:14:05 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 175828 ms exceeds timeout 120000 ms
26/01/31 20:14:05 WARN SparkContext: Killing executors is not supported by current scheduler.
26/01/31 20:14:11 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:359)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$